<a href="https://colab.research.google.com/github/jonlwowski012/StockSellRulesChecker/blob/master/StockScreener/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! git clone https://github.com/jonlwowski012/StockSellRulesChecker.git
! pip install -r StockSellRulesChecker/StockScreener/requirements.txt

Cloning into 'StockSellRulesChecker'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 44 (delta 14), reused 26 (delta 6), pack-reused 0
Unpacking objects: 100% (44/44), done.
     |████████████████████████████████| 9.5MB 8.0MB/s 
     |████████████████████████████████| 1.3MB 42.5MB/s 
     |████████████████████████████████| 143kB 48.0MB/s 
     |████████████████████████████████| 296kB 53.7MB/s 
  Created wheel for finviz: filename=finviz-1.3.4-cp37-none-any.whl size=15193 sha256=779c9abcb716745184a0e92f36ba0cb4d7e8e2cd4692bf613e8e5149d51649ad
  Stored in directory: /root/.cache/pip/wheels/6d/fa/7c/4cc075d8af3f8ed083dd112b1a2b09888b9a9a2964c907291f
  Created wheel for stockquotes: filename=stockquotes-2.0.1-cp37-none-any.whl size=4472 sha256=b599081a42bb9cd730199f784bc500c14e32735bd38a667c59ff488ca6ace996
  Stored in directory: /root/.cache/pip/wheels/b6/5a/fa/c250a28690344108071fb9b467

In [1]:
from finviz.screener import Screener
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
import datetime
import stockquotes
import pandas as pd
import tqdm
nest_asyncio.apply()

In [2]:
eps_5year_over20pct = True
eps_qoq_over20pct = True
eps_yoy_over20pct = True
sales_5years_over20pct = True
sales_qoq_over20pct = True
curr_vol_over200 = True

filters = []
if eps_5year_over20pct:
  filters.append('fa_eps5years_o20')
if eps_qoq_over20pct:
  filters.append('fa_epsqoq_o20')
if eps_yoy_over20pct:
  filters.append('fa_epsyoy_o20')
if sales_5years_over20pct:
  filters.append('fa_sales5years_o20')
if sales_qoq_over20pct:
  filters.append('fa_salesqoq_o20')
if curr_vol_over200:
  filters.append('sh_curvol_o200')

stock_list = Screener(filters=filters, table='Performance', order='price')  

In [5]:
print(stock_list)

No. | Ticker | Perf Week | Perf Month | Perf Quart | Perf Half | Perf Year | Perf YTD | Volatility W | Volatility M | Recom | Avg Volume | Rel Volume | Price  | Change | Volume 
--- | ------ | --------- | ---------- | ---------- | --------- | --------- | -------- | ------------ | ------------ | ----- | ---------- | ---------- | ------ | ------ | -------
1   | WKHS   | 2.08%     | -53.88%    | -23.37%    | -35.96%   | 1035.37%  | -15.62%  | 11.30%       | 19.52%       | 2.40  | 21.54M     | 0.58       | 16.00  | -4.13% | 772,275
2   | VALE   | 1.06%     | -0.92%     | 2.84%      | 50.12%    | 123.28%   | 3.46%    | 2.48%        | 3.67%        | 2.00  | 29.30M     | 0.28       | 17.17  | -0.09% | 513,988
3   | GNMK   | 19.98%    | 5.10%      | 84.41%     | 90.59%    | 426.43%   | 63.70%   | 9.92%        | 7.98%        | 1.90  | 1.95M      | 6.81       | 23.90  | -0.02% | 824,516
4   | EXPI   | 21.33%    | -28.42%    | 66.32%     | 165.73%   | 1133.41%  | 76.65%   | 11.83%       | 13.20% 

In [3]:
def moving_average(yahoo_df, days):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=days)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())


screened_stocks = {}
for stock in tqdm.tqdm(stock_list):
    screened_stocks[stock['Ticker']] = {}
    
    finviz_stats = finviz.get_stock(stock['Ticker'])
    try:
        yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
    except:
        continue
    SMA200_value = moving_average(yahoo_df, days=200)
    SMA150_value = moving_average(yahoo_df, days=150)
    SMA50_value = moving_average(yahoo_df, days=50)
    SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))
    SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))
    prev_close = float(finviz_stats['Prev Close'].replace("$",""))
    week52_high, week52_low = week52_low_high(yahoo_df)
    
    screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
    screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
    screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
    screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
    screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
    screened_stocks[stock['Ticker']]['prev_close'] = prev_close
    screened_stocks[stock['Ticker']]['week52_high'] = week52_high
    screened_stocks[stock['Ticker']]['week52_low'] = week52_low
    
    # Positive 200d MA positive
    if SMA200_percent > 0:
        SMA200_slope_rule = True
    else:
        SMA200_slope_rule = False
    screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
        
    # 150d MA greater than 200d MA
    if SMA150_value > SMA200_value:
        SMA150_greater_SMA200_rule = True
    else:
        SMA150_greater_SMA200_rule = False
    screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
        
    # 50d MA greater than 150d MA
    if SMA50_value > SMA150_value:
        SMA50_greater_SMA150_rule = True
    else:
        SMA50_greater_SMA150_rule = False
    screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
        
    # Close above 50d MA
    if prev_close > SMA50_value:
        close_greater_SMA50_rule = True
    else:
        close_greater_SMA50_rule = False
    screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
        
    # 52 week high low span rule
    if 0.75*week52_high > 1.25*week52_low:
        week52_span_rule = True
    else:
        week52_span_rule = False
    screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
    
    # Close above 52 week high - 25%
    if prev_close > 0.75*week52_high:
        close_above_52weekhigh_rule = True
    else:
        close_above_52weekhigh_rule = False
    screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule

output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

    


100%|██████████| 8/8 [00:05<00:00,  1.57it/s]


In [4]:
df_out

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,prev_close,week52_high,week52_low,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule
0,WKHS,23.664228,23.516520,26.832714,-20.70,-38.53,16.69,41.340000,1.540000,False,False,True,False,True,False
1,VALE,14.532463,15.683971,17.374143,29.84,-1.48,17.19,18.940001,6.580000,True,True,True,False,True,True
2,GNMK,14.701029,15.275196,18.310286,61.92,37.35,23.90,23.980000,3.560000,True,True,True,True,True,True
3,EXPI,36.347261,40.825049,60.880858,86.15,-4.02,55.75,80.525002,3.505000,True,True,True,False,True,False
4,JD,86.092573,88.930784,94.530857,6.17,-10.58,85.76,106.879997,37.209999,True,True,True,False,True,True
5,BABA,267.042574,261.017157,251.518858,-12.70,-8.90,226.93,317.140015,176.339996,False,False,False,False,True,False
6,FB,269.894963,270.807401,266.282142,5.43,4.12,279.28,303.910004,146.960007,True,True,False,True,True,True
7,NVDA,534.090149,536.599119,545.585429,5.55,-3.43,531.65,613.210022,202.820007,True,True,True,False,True,True


In [ ]:
df_out.to_csv("screened_stocks.csv")